### Pacotes necessários

In [ ]:
import pandas as pd
import sys
import scipy
import matplotlib 
import seaborn
import numba
import gmpy2
import pysubgroup as ps
import numpy as np
import math
sys.path.append('./projeto_aprendizado_descritivo')
sys.path
from src.util.results2folder import attach_results,print2folder
from _classes import SSDC

### Leitura do banco de dados

In [ ]:
df = pd.read_csv('./data/dengue_internacao.csv', sep=',')

In [ ]:
# selecionando as colunas desejadas
df_selecionado = df[['ano', 'data_notificacao', 'sigla_uf_notificacao', 'data_primeiros_sintomas', 'idade_paciente', 
                     'sexo_paciente', 'raca_cor_paciente', 'escolaridade_paciente', 'gestante_paciente', 
                     'possui_doenca_autoimune', 'possui_diabetes', 'possui_doencas_hematologicas', 
                     'possui_hepatopatias', 'possui_doenca_renal', 'possui_hipertensao', 
                     'possui_doenca_acido_peptica', 'apresenta_febre','apresenta_cefaleia','apresenta_exantema','apresenta_dor_costas','apresenta_mialgia','apresenta_vomito','apresenta_conjutivite','apresenta_dor_retroorbital',
                     'apresenta_artralgia','apresenta_artrite','apresenta_leucopenia','apresenta_petequias',
                     'data_internacao','classificacao_final','evolucao_caso', 'data_obito', 'data_encerramento']]

df_selecionado = df_selecionado[df_selecionado['ano'] >= 2023]


### Tratamento do banco de dados

In [ ]:
# transformando atributo nominal data_obito para binário houve_obito
df_selecionado['houve_obito'] = df_selecionado['data_obito'].apply(lambda x: 0 if pd.isna(x) else 1)

# remoção data_obito
df_selecionado = df_selecionado.drop(columns=['data_obito'])

In [ ]:
# removendo as linhas onde não houveram datas de primeiros sintomas ou data da notificação.
df_selecionado = df_selecionado.dropna(subset=['data_primeiros_sintomas', 'data_notificacao'])

df_selecionado['data_notificacao'] = pd.to_datetime(df_selecionado['data_notificacao'], errors='coerce')
df_selecionado['data_primeiros_sintomas'] = pd.to_datetime(df_selecionado['data_primeiros_sintomas'], errors='coerce')
# criando a coluna 'tempo_internacao'
df_selecionado['intervalo_busca_atendimento'] = df_selecionado.apply(
    lambda row: (row['data_notificacao'] - row['data_primeiros_sintomas']).days if pd.notna(row['data_primeiros_sintomas']) else 0,
    axis=1
)

In [ ]:
# removendo as linhas onde não houveram internações ou data de encerramento.
df_selecionado = df_selecionado.dropna(subset=['data_internacao', 'data_encerramento'])

# convertendo as colunas para datetime
df_selecionado['data_encerramento'] = pd.to_datetime(df_selecionado['data_encerramento'], errors='coerce')
df_selecionado['data_internacao'] = pd.to_datetime(df_selecionado['data_internacao'], errors='coerce')

# criando a coluna 'intervalo_sintoma_internacao' que é a data dos primeiros sintomas até a data que ela foi internada.
df_selecionado['intervalo_sintoma_internacao'] = df_selecionado.apply(
    lambda row: (row['data_internacao'] - row['data_primeiros_sintomas']).days if pd.notna(row['data_internacao']) else 0,
    axis=1
)

# criando a coluna 'intervalo_internacao'
df_selecionado['intervalo_internacao'] = df_selecionado.apply(
    lambda row: (row['data_encerramento'] - row['data_internacao']).days if pd.notna(row['data_internacao']) else 0,
    axis=1
)

df_selecionado = df_selecionado.query("intervalo_internacao >= 0 and intervalo_internacao <= 90 and" + " intervalo_sintoma_internacao >= 0 and intervalo_sintoma_internacao <= 90 and"
              + " intervalo_busca_atendimento >= 0 and intervalo_busca_atendimento <= 90")

In [ ]:
# prompt: na variável, idade_paciente, retirar manter os valores depois de "-" e transformar em inteiro
df_selecionado['idade_paciente'] = df_selecionado['idade_paciente'].str.split('-').str[1]
df_selecionado['idade_paciente'] = df_selecionado['idade_paciente'].str.extract('(\d+)').astype(int)

In [ ]:
# drop das variáveis data_notificacao, data_primeiros_sintomas, data_internacao e data_encerramento
df_selecionado = df_selecionado.drop(['data_notificacao', 'data_primeiros_sintomas', 'data_internacao', 'data_encerramento'], axis=1)

In [ ]:
# recodificando as variáveis categóricas
raca_cor_map = {
    1:"Branca",
    2:"Preta",
    3:"Amarela",
    4:"Parda",
    5:"Indígena",
    9:"Ignorado"
}
df_selecionado["raca_cor_paciente"] = df_selecionado["raca_cor_paciente"].map(raca_cor_map)

escolaridade_map = {
    0: "Analfabeto",
    1: "1ª a 4ª série incompleta do EF",
    2: "4ª série completa do EF",
    3: "5ª A 8ª série incomplet do EF",
    4: "Ensino fundamental incompleto",
    5: "Ensino médio incompleto",
    6: "Ensino médio completo",
    7: "Educação superior incompleta",
    8: "Educação superior completa",
    9: "Ignorado",
    10: "Não se Aplica"
}

df_selecionado["escolaridade_paciente"] = df_selecionado["escolaridade_paciente"].map(escolaridade_map)

gestante_map = {
    1:"1ºTrimestre", 2:"2ºTrimestre", 3:"3ºTrimestre", 4:"Idade gestacional Ignorada", 5:"Não", 6:"Não se aplica",9:"Ignorado"
}

df_selecionado["gestante_paciente"] = df_selecionado["gestante_paciente"].map(gestante_map)

dicotomia_map = {
    1: "Sim",
    2: "Não"
}

col_dicotomia = ['possui_doenca_autoimune', 'possui_diabetes', 'possui_doencas_hematologicas', 
                     'possui_hepatopatias', 'possui_doenca_renal', 'possui_hipertensao', 
                     'possui_doenca_acido_peptica', 'apresenta_febre','apresenta_cefaleia','apresenta_exantema','apresenta_dor_costas','apresenta_mialgia','apresenta_vomito','apresenta_conjutivite','apresenta_dor_retroorbital',
                     'apresenta_artralgia','apresenta_artrite','apresenta_leucopenia','apresenta_petequias']

for col in col_dicotomia:
    df_selecionado[col] = df_selecionado[col].map(dicotomia_map)

classificacao_map={
    5: "Descartado",
    10: "Dengue", 
    11: "Dengue com Sinais de Alarme",
    12: "Dengue Grave", 
    13: "Chikungunya"
}

df_selecionado['classificacao_final'] = df_selecionado['classificacao_final'].map(classificacao_map).fillna(np.nan)

evolucao_map={
    1:"Cura",
    2:"Óbito pelo agravo", 
    3:"Óbito por outras causas",
    4:"Óbito em investigação", 
    9:"Ignorado",
}

df_selecionado['evolucao_caso']=df_selecionado['evolucao_caso'].map(evolucao_map).fillna(np.nan)
df_selecionado['houve_obito'] = df_selecionado['houve_obito'].map({1: 'sim', 0: 'não'})

df_selecionado

In [ ]:
## retirando os resultados NaN do banco de dados
df_selecionado = df_selecionado.dropna()


In [ ]:
# transformar variáveis nominais em categóricas
df_aux = df_selecionado
categoricas_var = ['sigla_uf_notificacao','sexo_paciente','raca_cor_paciente','escolaridade_paciente','gestante_paciente','possui_doenca_autoimune', 'possui_diabetes', 'possui_doencas_hematologicas', 
                     'possui_hepatopatias', 'possui_doenca_renal', 'possui_hipertensao', 
                     'possui_doenca_acido_peptica', 'apresenta_febre','apresenta_cefaleia','apresenta_exantema','apresenta_dor_costas','apresenta_mialgia','apresenta_vomito','apresenta_conjutivite','apresenta_dor_retroorbital',
                     'apresenta_artralgia','apresenta_artrite','apresenta_leucopenia','apresenta_petequias','classificacao_final','evolucao_caso','houve_obito']

for col in categoricas_var:
    df_selecionado[col]=pd.Categorical(df_selecionado[col])

df_selecionado

### Teste intervalo_internacao BeamSearch

In [ ]:
# Carregar banco df_selecionado

data = df_selecionado


#target = ps.BinaryTarget ('sobrevivencia', True)
target2 = ps.NumericTarget('intervalo_internacao')
searchspace = ps.create_selectors(data, ignore=['intervalo_internacao'])
task = ps.SubgroupDiscoveryTask (
    data,
    target2,
    searchspace,
    result_set_size=60,
    depth=3,
    qf=ps.StandardQFNumeric(1.0))
result = ps.BeamSearch(beam_width=65).execute(task)
print(result.to_dataframe())

In [ ]:
result_beam_search = result.to_dataframe()
#Calculando o z-score do Beam Search
def zscore(n, mu_s, mu_d, sig_d):
    value = ((float(n)**0.5)*(float(mu_s)-mu_d))/sig_d
    return abs(value)
mu = result_beam_search['mean_dataset'].values[0]
sig = result_beam_search['std_dataset'].values[0]
result_beam_search['z-score'] = result_beam_search.apply(lambda x: zscore(x['size_sg'], x['mean_sg'], mu, sig), axis=1)

### Teste intervalo_internacao DFS

In [ ]:
# Carregar banco df_selecionado

data = df_selecionado


#target = ps.BinaryTarget ('sobrevivencia', True)
target2 = ps.NumericTarget('intervalo_internacao')
searchspace = ps.create_selectors(data, ignore=['intervalo_internacao'])
task = ps.SubgroupDiscoveryTask (
    data,
    target2,
    searchspace,
    result_set_size=60,
    depth=2,
    qf=ps.StandardQFNumeric(1.0))
result = ps.DFS().execute(task)
print(result.to_dataframe())

In [ ]:
result_dfs=result.to_dataframe()
#Calculando o z-score do DFS
result_dfs['z-score'] = result_dfs.apply(lambda x: zscore(x['size_sg'], x['mean_sg'], mu, sig), axis=1)

### 	Teste intervalo_internação SSDC

In [ ]:
df_selecionado.dtypes

In [ ]:
task_name = "discovery"
target_type = "numeric"

# load class and fit to data
model = SSDC(task = task_name,max_depth=3, beam_width=25,max_rules=20,n_cutpoints=3)
model.fit(df_selecionado)

In [ ]:
print(model)
pd.DataFrame(model.antecedent_description)

In [ ]:
model2 = SSDC(task = task_name, max_depth=4, beam_width=25,max_rules=20,n_cutpoints=3)
model2.fit(df_selecionado)

In [ ]:
model3 = SSDC(task = task_name,max_depth=3, beam_width=25,max_rules=20,n_cutpoints=5)
model3.fit(df_selecionado)

In [ ]:
model4 = SSDC(task = task_name,max_depth=4, beam_width=25,max_rules=20,n_cutpoints=5)
model4.fit(df_selecionado)

In [ ]:
model5 = SSDC(task = task_name,max_depth=3, beam_width=50,max_rules=20,n_cutpoints=3)
model5.fit(df_selecionado)

In [ ]:
model6 = SSDC(task = task_name,max_depth=4, beam_width=50,max_rules=20,n_cutpoints=3)
model6.fit(df_selecionado)


### Estatísticas de Qualidade por modelo no SSDC

#### Modelo 1

In [ ]:
model_estat = pd.DataFrame(model.statistic_rules)

In [ ]:
df_model = pd.DataFrame(model.antecedent_description, columns=['Subgrupros'])

In [ ]:
result_model = pd.concat([df_model, model_estat], axis=1)

In [ ]:
result_model['z-score'] = result_model.apply(lambda x: zscore(x['usage'], x['mean'], mu, sig), axis=1)
result_model.sort_values('z-score', ascending=False, inplace=True)

In [ ]:
result_model

#### Modelo 2

In [ ]:
model_estat2 = pd.DataFrame(model2.statistic_rules)
df_model2 = pd.DataFrame(model2.antecedent_description, columns=['Subgrupros'])
result_model2 = pd.concat([df_model2, model_estat2], axis=1)
result_model2['z-score'] = result_model2.apply(lambda x: zscore(x['usage'], x['mean'], mu, sig), axis=1)
result_model2.sort_values('z-score', ascending=False, inplace=True)

#### Modelo 3

In [ ]:
model_estat3 = pd.DataFrame(model3.statistic_rules)
df_model3 = pd.DataFrame(model3.antecedent_description, columns=['Subgrupros'])
result_model3 = pd.concat([df_model3, model_estat3], axis=1)
result_model3['z-score'] = result_model3.apply(lambda x: zscore(x['usage'], x['mean'], mu, sig), axis=1)
result_model3.sort_values('z-score', ascending=False, inplace=True)

#### Modelo 4

In [ ]:
model_estat4 = pd.DataFrame(model4.statistic_rules)
df_model4 = pd.DataFrame(model4.antecedent_description, columns=['Subgrupros'])
result_model4 = pd.concat([df_model4, model_estat4], axis=1)
result_model4['z-score'] = result_model4.apply(lambda x: zscore(x['usage'], x['mean'], mu, sig), axis=1)
result_model4.sort_values('z-score', ascending=False, inplace=True)

#### Modelo 5

In [ ]:
model_estat5 = pd.DataFrame(model5.statistic_rules)
df_model5 = pd.DataFrame(model5.antecedent_description, columns=['Subgrupros'])
result_model5 = pd.concat([df_model5, model_estat5], axis=1)
result_model5['z-score'] = result_model5.apply(lambda x: zscore(x['usage'], x['mean'], mu, sig), axis=1)
result_model5.sort_values('z-score', ascending=False, inplace=True)

#### Modelo 6

In [ ]:
model_estat6 = pd.DataFrame(model6.statistic_rules)
df_model6 = pd.DataFrame(model6.antecedent_description, columns=['Subgrupros'])
result_model6 = pd.concat([df_model6, model_estat6], axis=1)
result_model6['z-score'] = result_model6.apply(lambda x: zscore(x['usage'], x['mean'], mu, sig), axis=1)
result_model6.sort_values('z-score', ascending=False, inplace=True)

### Salvando os experimentos

In [ ]:
experimentos = 'experimentos_sd.xlsx'

with pd.ExcelWriter(experimentos) as writer:
    result_model.to_excel(writer, sheet_name='SSD_modelo_1', index=False)
    result_model2.to_excel(writer, sheet_name='SSD_modelo_2', index=False)
    result_model3.to_excel(writer, sheet_name='SSD_modelo_3', index=False)
    result_model4.to_excel(writer, sheet_name='SSD_modelo_4', index=False)
    result_model5.to_excel(writer, sheet_name='SSD_modelo_5', index=False)
    result_model6.to_excel(writer, sheet_name='SSD_modelo_6', index=False)
    result_beam_search.to_excel(writer, sheet_name='Beam_Search', index=False)
    result_dfs.to_excel(writer, sheet_name='DFS', index=False)